# Extração de dados de liberações de recursos dos programas do FNDE

Importando os pacotes necessários

In [ ]:
import urllib.request

In [ ]:
import re

In [ ]:
import pandas as pd

In [ ]:
import ssl

Desabilitando a verificação do contexto

In [ ]:
context = ssl._create_unverified_context()

Criando a lista para armazenar os anos da pesquisa

In [ ]:
anos = []

Definindo o ano inicial da pesquisa

In [ ]:
primeiro_ano = 2018

Definindo o ano final da pesquisa

In [ ]:
ultimo_ano = 2018

Carregando a lista com os anos a serem pesquisados

In [ ]:
for ano in range(primeiro_ano, ultimo_ano + 1):
    anos.append(ano)

Selecionando os códigos dos programas<br>
<br>
Exemplos de códigos que podem ser utilizados<br>
C7 - ALIMENTAÇÃO ESCOLAR (PROG.NACIONAL DE ALIMENTAÇÃO ESCOLAR)<br>
02 - PDDE (PROGRAMA DINHEIRO DIRETO NA ESCOLA )<br>
0A - PDDE ESTRUTURA (ÁGUA E ESGOTAMENTO SANITÁRIO, ESCOLA DO CAMPO, ESCOLA ACESSÍVEL E PDE ESCOLA)<br>
0B - PDDE QUALIDADE ( ENSINO MÉDIO INOVADOR, MAIS CULTURA, ESC.DE FRONTEIRA, ATLETA NA ESCOLA, ESC.SUSTENTÁVEL)<br>
A3 - PDDE-EDUCAÇÃO ESPECIAL (PROGRAMA DINHEIRO DIRETO NA ESCOLA-EDUCAÇÃO ESPECIAL)<br>
Z9 - PDDE-EDUCAÇÃO INTEGRAL (PROGRAMA DINHEIRO DIRETO NA ESCOLA EDUCAÇÃO  INTEGRAL)<br>
D8 - PNATE (PROGRAMA NACIONAL DE APOIO AO TRANSP DO ESCOLAR)

In [ ]:
codigos = ['C7', '02']

Definindo o padrão da string da url da pesquisa

In [ ]:
padrao_url = 'https://www.fnde.gov.br/sigefweb/index.php/liberacoes?ano={}&programa={}&cnpj=&estado=0&tipoEntidade=todos&confirmar=Confirmar'

Criando uma lista para armazenar as urls da pesquisa

In [ ]:
urls = []

Carregando a lista com as urls a serem pesquisadas

In [ ]:
for ano in reversed(anos):
    for codigo in codigos:
        urls.append(padrao_url.format(ano, codigo))

Conferindo as urls carregadas na lista

In [ ]:
len(urls)

In [ ]:
urls

Criando uma lista para carregar as esferas pesquisadas em cada url

In [ ]:
lista_esferas = []

Carregando a lista de esferas a partir das urls

In [ ]:
for url in urls:
    with urllib.request.urlopen(url, context=context) as response:
       html = response.read().decode('ansi')
    esferas = re.findall(r'<tr><td><a href=\"(.*?)\">', html)
    lista_esferas = lista_esferas + esferas

Conferindo a lista de esferas carregada

In [ ]:
len(lista_esferas)

In [ ]:
lista_esferas

Para excluir uma das esferas caso não haja necessidade de pesquisá-la.

In [ ]:
lista_esferas.remove('/sigefweb/index.php/liberacoes/resultado-parcial/ano/2018/programa/02/tipoEntidade/todos/esfera/estadual')

Definindo a url do site do FNDE

In [ ]:
site_fnde = 'https://www.fnde.gov.br'

Criando as listas das urls das entidades de cada programa e das UF de cada entidade

In [ ]:
lista_entidades = []

Carregando a lista de entidades de cada programa

In [ ]:
for esfera in lista_esferas:
    with urllib.request.urlopen(site_fnde + esfera, context=context) as response:
       html = response.read().decode('ansi')
    entidades = re.findall(r'<tr><td><a href=\"(.*?)\">', html)
    lista_entidades = lista_entidades + entidades

Conferindo o tamanho da lista de entidades

In [ ]:
len(lista_entidades)

Criando as listas onde serão armazenadas as informações das entidades pesquisadas

In [ ]:
lista_dados = []

In [ ]:
lista_cnpjs = []

In [ ]:
lista_nomes_entidades = []

In [ ]:
lista_ufs = []

In [ ]:
lista_municipios = []

In [ ]:
lista_totais = []

Carregando as listas com as informações de cada entidade. As expressões regulares foram construídas com a ferramenta disponível no site: http://www.regexlab.com/wild2regex, https://pythex.org/ e https://regex101.com/

In [ ]:
for i, entidade in enumerate(lista_entidades[6121:10000]):
    with urllib.request.urlopen(site_fnde + entidade, context=context) as response:
        html = response.read().decode('ansi')
    dados = re.findall(r'\t<td>(.*?)</td>', html)
    lista_cnpjs = lista_cnpjs + [dados[0]]
    lista_nomes_entidades = lista_nomes_entidades + [dados[1]]
    lista_ufs = lista_ufs + [dados[2]]
    lista_municipios = lista_municipios + [dados[3]]
    totais = re.findall(r'<tfoot><tr><td>&nbsp;</td><td>Total: </td><td>(.*?)</td>', html)
    if len(totais) == 0:
        totais = ['0,00']
    lista_totais = lista_totais + totais

Conferindo os tamanhos das listas com os dados

In [ ]:
len(lista_cnpjs)

In [ ]:
len(lista_nomes_entidades)

In [ ]:
len(lista_ufs)

In [ ]:
len(lista_municipios)

In [ ]:
len(lista_totais)

Criando um dicionário com as listas de dados

In [ ]:
entidades_dict = {'CNPJ': lista_cnpjs, 'ENTIDADE': lista_nomes_entidades, 'UF': lista_ufs, 'MUNICIPIO': lista_municipios, 'TOTAL': lista_totais}

Transformando o dicionário em um banco de dados

In [ ]:
entidades_uf = pd.DataFrame.from_dict(entidades_dict)

Exportando o banco de dados para planilha do Excel

In [ ]:
entidades_uf.to_excel('liberacoes.xlsx', index=False)